In [26]:
from datasets import load_dataset
import random
from transformers import pipeline
import torch
from sentence_transformers import SentenceTransformer, util
from sklearn.model_selection import train_test_split
import pandas as pd
#from tqdm import tqdm
from tqdm.notebook import tqdm
import evaluate

In [2]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
#os.environ["TORCH_USE_CUDA_DSA"] = "1"

In [3]:
#dataset = load_dataset('code-search-net/code_search_net')
dataset = load_dataset("Fsoft-AIC/the-vault-function", split_set=["test"], trust_remote_code=True)

README.md:   0%|          | 0.00/11.3k [00:00<?, ?B/s]

the-vault-function.py:   0%|          | 0.00/15.0k [00:00<?, ?B/s]

python-00000-of-00001.parquet:   0%|          | 0.00/30.3M [00:00<?, ?B/s]

c-00000-of-00001.parquet:   0%|          | 0.00/25.9M [00:00<?, ?B/s]

c_sharp-00000-of-00001.parquet:   0%|          | 0.00/18.5M [00:00<?, ?B/s]

...

javascript-00000-of-00001.parquet:   0%|          | 0.00/26.4M [00:00<?, ?B/s]

php-00000-of-00001.parquet:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

ruby-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

rust-00000-of-00001.parquet:   0%|          | 0.00/29.0M [00:00<?, ?B/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:
sample_df = pd.DataFrame(dataset['test'])
sample_df.columns

Index(['hexsha', 'repo', 'path', 'license', 'language', 'identifier',
       'return_type', 'original_string', 'original_docstring', 'docstring',
       'docstring_tokens', 'code', 'code_tokens', 'short_docstring',
       'short_docstring_tokens', 'comment', 'parameters', 'docstring_params'],
      dtype='object')

In [6]:
dataset_sample = sample_df.groupby('language', group_keys=False).sample(n=20)
dataset_sample.columns

Index(['hexsha', 'repo', 'path', 'license', 'language', 'identifier',
       'return_type', 'original_string', 'original_docstring', 'docstring',
       'docstring_tokens', 'code', 'code_tokens', 'short_docstring',
       'short_docstring_tokens', 'comment', 'parameters', 'docstring_params'],
      dtype='object')

# Testing the Baseline model

Here we define the system prompt for the Llama 2 model.

In [7]:
def llm_prompt(language, comment, code):
    prompt_gemma = \
    '''You are a helpful agent designed to simplify code documentation for beginner programmers.
    You will be provided with a block of {LANGUAGE} code and the existing doucmentation that accompanies it.
    Using the provided code as context, give a simplified explanation of the code so that it is understandable
    to beginner programmers. Output absolutely nothing else besides the simplified explanation.
    Make sure to keep any documentation formatting codes present in the simplified explanation.
    If you feel that the existing documentation is simple enough and meaning would be lost by simplifying
    it further, feel free to keep the documentation as is. Here is the original documentation and code: \nDocumenation:\n{Documentation}\n\nCode:\n{CODE}'''.format(LANGUAGE=language, Documentation=comment, CODE=code)
    
    return prompt_gemma

Creating the pipeline for the Gemma 3 model using the HuggingFace transformers library. Modified from the example here: https://huggingface.co/docs/transformers/v4.51.3/en/model_doc/gemma#gemma

In [9]:
pipe_gemma = pipeline(
    "text2text-generation",
    model="google/gemma-3-4b-it",
    torch_dtype=torch.float32,
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
)
"""pipe_gemma = pipeline(
    "text2text-generation",
    model="google/gemma-3-4b-it",
    torch_dtype=torch.float32,
    device="cpu"
)"""

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

Device set to use cuda
The model 'Gemma3ForConditionalGeneration' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'Qwen2AudioForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditi

'pipe_gemma = pipeline(\n    "text2text-generation",\n    model="google/gemma-3-4b-it",\n    torch_dtype=torch.float32,\n    device="cpu"\n)'

In [10]:
eval_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
lan = dataset['test'][0]['language']
code = dataset['test'][0]['original_string']
doc = dataset['test'][0]['original_docstring']
print(f"Original Documentation:\n{dataset['test'][0]['original_docstring']}\n")
print(f"Code:\n{dataset['test'][0]['original_string']}\n")
print("***********************Result************************************")
print(pipe_gemma(llm_prompt(lan, doc, code), pad_token_id=pipe_gemma.tokenizer.eos_token_id, max_new_tokens=150)[0]['generated_text'])

Original Documentation:
Build the library mappings tables.

Code:
def build_mapping_tables(app):
    """Build the library mappings tables."""
    env = Environment(loader=FileSystemLoader(f"{DIR_PATH}"))
    template_file = env.get_template("table_template.j2")

    LIST_OF_MAP_DICTS = []
    for attr in dir(lib_mapper):
        if (attr.endswith("MAPPER_REVERSE") or attr.endswith("_MAPPER")) and not (
            attr.startswith("_") or attr.startswith("NETMIKO") or attr.startswith("MAIN")
        ):
            LIST_OF_MAP_DICTS.append(attr)

    for dict_name in LIST_OF_MAP_DICTS:
        lib_name = dict_name.split("_")[0]
        filename = f"{lib_name}_reverse" if "REVERSE" in dict_name else lib_name
        headers = ["NORMALIZED", lib_name] if "REVERSE" in dict_name else [lib_name, "NORMALIZED"]
        rendered_template = template_file.render(lib_names=headers, mappings=getattr(lib_mapper, dict_name))
        with open(f"{DIR_PATH}/netutils/lib_mapping/{filename}_table.rst", "w

In [19]:
generated_documents = []
gemma_semantic_similarities_untrained = []
gemma_metrics_untrained = evaluate.combine(['rouge', 'meteor'])

for instance in tqdm(dataset_sample.itertuples()):
    language = instance.language
    original_doc = instance.original_docstring
    orignal_code = instance.original_string

    message = llm_prompt(language, original_doc, orignal_code)

    result = pipe_gemma(message, pad_token_id=pipe_gemma.tokenizer.eos_token_id, max_new_tokens=150)[0]['generated_text']

    result = result.replace(message, "")

    embedding_original = eval_model.encode(instance.original_docstring, convert_to_tensor=True)
    embedding_predicted = eval_model.encode(result, convert_to_tensor=True)

    gemma_semantic_similarities_untrained.append(util.pytorch_cos_sim(embedding_original, embedding_predicted).item())
    gemma_metrics_untrained.add(predictions=result, references=instance.original_docstring)

[nltk_data] Downloading package wordnet to /home/adeniji/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/adeniji/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/adeniji/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


0it [00:00, ?it/s]

In [21]:
print("Untrained Gemma 3: \n")
sims_untrianed = pd.DataFrame(gemma_semantic_similarities_untrained)
sims_untrianed.to_excel('results/Semantic_Similarities_Gemma_untrained.xlsx')
sims_untrianed.describe()

Untrained Gemma 3: 



,0
count,200.000000
mean,0.588644
std,0.214955
min,-0.035175
25%,0.497760
50%,0.633050
75%,0.726025
max,1.000000


In [22]:
import json
with open('results/rouge_meteor_gemma_untrained.json', 'w') as file:
    mr = gemma_metrics_untrained.compute()
    json.dump(mr, file, indent=4)

# Testing the FineTuned model

Here we Import the training examples from an excel file.

In [25]:
train_df = pd.read_excel("self_training_annotated.xlsx", sheet_name="Sheet1", usecols=[5, 8, 9, 16])
training_sample = train_df.groupby('language', group_keys=False).sample(n=5)
training_sample.columns

Index(['language', 'original_string', 'original_docstring',
       'modified_short_docstring'],
      dtype='object')

In [24]:
def training_prompt(language, documentation, code, modified):
    prompt_gemma = \
    '''You are a helpful agent designed to help beginner programmers learn the fundamentals of programming.
    You will be provided with a block of {LANGUAGE} code and the existing doucmentation that accompanies it.
    You will also be provided a simplified version of the documentation. Using the provided code as context,
    your goal is to understand how the documentation is simplified while being contextually relevant to the 
    original code. Output absolutely nothing. If you feel that the simplified documentation is simple and contextually 
    relevant learn from the example. If you do feel  the simplified documentaiton is not simple or not contextually relevant
    forget about the example. Here is the original documentation and code:\n 
    Code:\n{CODE}\n\nDocumentation:\n{DOCUMENTATION}\n\nModified documentation:\n{MODIFIED}'''.format(LANGUAGE=language, DOCUMENTATION=documentation, CODE=code, MODIFIED=modified)

In [ ]:
for instance in tqdm(training_sample.itertuples()):
    language = instance.language
    original_doc = instance.original_docstring
    orignal_code = instance.original_string
    mod = instance.modified_short_docstring

    message = training_prompt(language, original_doc, orignal_code, mod)

    pipe_gemma(message, pad_token_id=pipe_gemma.tokenizer.eos_token_id, max_new_tokens=45)

0it [00:00, ?it/s]

In [ ]:
gemma_semantic_similarities_trained = []
gemma_metrics_trained = evaluate.combine(['rouge', 'meteor'])


for instance in tqdm(dataset_sample.itertuples()):
    language = instance.language
    original_doc = instance.original_docstring
    orignal_code = instance.original_string

    message = llm_prompt(language, original_doc, orignal_code)

    result = pipe_gemma(message, pad_token_id=pipe_gemma.tokenizer.eos_token_id, max_new_tokens=30)[0]['generated_text']

    embedding_original = eval_model.encode(instance.original_docstring, convert_to_tensor=True)
    embedding_predicted = eval_model.encode(result, convert_to_tensor=True)

    gemma_semantic_similarities_trained.append(util.pytorch_cos_sim(embedding_original, embedding_predicted).item())
    gemma_metrics_trained.add(predictions=result, references=instance.original_docstring)

[nltk_data] Downloading package wordnet to /home/adeniji/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/adeniji/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/adeniji/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


0it [00:00, ?it/s]

# Summary statistics results
Semantic Similarity

In [19]:
print("Trained Gemma 3: \n")
sims_trained = pd.DataFrame(gemma_semantic_similarities_trained)
sims_trained.to_excel('results/Semantic_Similarities_Gemma_trained.xlsx')
sims_trained.describe()

Trained Gemma 3: 



,0
count,200.000000
mean,0.504685
std,0.141379
min,0.051231
25%,0.409741
50%,0.516998
75%,0.609724
max,0.840501


ROUGE AND METEOR

In [20]:
with open('results/rouge_meteor_gemma_trained.json', 'w') as file:
    mr = gemma_metrics_trained.compute()
    json.dump(mr, file, indent=4)
